### Imports

In [3]:
import tensorflow as tf
import numpy as np
from PIL import Image
import nbimporter
import Model

### Data init

In [8]:
X_train = np.load("/root/U-netKeras/data/clouddata_normalized.npy")
Y_train = np.load("/root/U-netKeras/data/mask.npy")